In [1]:
using ModelingToolkit, OrdinaryDiffEq
using BoundaryValueDiffEq, LinearAlgebra, Plots
using ModelingToolkit: t_nounits as t, D_nounits as D

In [2]:
@mtkmodel TWOFLUID begin
    @parameters begin
        # growth rate guess
        γ
    end

    @constants begin
        # mode numbers of interest
        m = 2
        n = 1

        # device parameters
        R0 = 1

        # poloidal wave numbers 
        ky = 0.25
        km = (m - 1) * ky
        k = (m) * ky
        kp = (m + 1) * ky

        # toroidal wave number
        kz = 1 / R0
        kn = n * kz

        # combined wave numbers
        Km2 = km^2 + kn^2
        K2 = k^2 + kn^2
        Kp2 = kp^2 + kn^2

        # Zeta-components
        ζz = 1
        ζy = 1

        # Lundquist number
        S = 100

        # Alfvénic Mach number
        M = S^(-1 / 2)
    end

    @variables begin
        ψm(t) = 0
        ψ(t) = 0
        ψp(t) = 0
        φm(t) = 0
        φ(t) = 0
        φp(t) = 0
        f(t)
    end

    @equations begin
        f ~ tanh(t)
        # m-1 mode equations
        γ * (D(D(φm)) - Km2 * φm) + (M / 2) * (D(D(D(φ))) - (Km2 - ζz^2 * ky^2) * D(φ)) ~ -ky * (f * (m - 1) - 1) * (D(D(ψm)) - Km2 * ψm - (D(D(f)) / f) * ψm)
        γ * ψm + (M / 2) * D(ψ) - ky * (f * (m - 1) - 1) * φm - (1 / S) * (D(D(ψm)) - Km2 * ψm) ~ 0

        # m mode equations
        γ * (D(D(φ)) - K2 * φ) + (M / 2) * (D(D(D(φp))) - (Kp2 - ky^2) * D(φp) + D(D(D(φm))) - (Kp2 - ky^2) * D(φm)) ~ -k * f * (D(D(ψ)) - K2 * ψ - (D(D(f)) / f) * ψ)
        γ * ψ + (M / 2) * (D(ψp) + D(ψm)) - k * f * φ - (1 / S) * (D(D(ψ)) - K2 * ψ) ~ 0

        # m+1 mode equations
        γ * (D(D(φp)) - Kp2 * φp) + (M / 2) * (D(D(D(φ))) - (Kp2 - ζz^2 * ky^2) * D(φ)) ~ -ky * (f * (m + 1) + 1) * (D(D(ψp)) - Kp2 * ψp - (D(D(f)) / f) * ψp)
        γ * ψp + (M / 2) * D(ψ) - ky * (f * (m + 1) + 1) * φp - (1 / S) * (D(D(ψp)) - Kp2 * ψp) ~ 0

    end
end

@mtkcompile tfmodel = TWOFLUID()

Model tfmodel:
Equations (15):
  15 standard: see equations(tfmodel)
Unknowns (15): see unknowns(tfmodel)
  ψp(t) [defaults to 0]
  φp(t) [defaults to 0]
  ψ(t) [defaults to 0]
  ψpˍt(t)
  ψm(t) [defaults to 0]
  ψˍt(t)
  ⋮
Parameters (17): see parameters(tfmodel)
  γ
  Km2 [defaults to km^2 + kn^2]
  ky [defaults to 0.25]
  kn [defaults to kz*n]
  km [defaults to ky*(-1 + m)]
  ζz [defaults to 1]
  ⋮
Observed (7): see observed(tfmodel)

In [3]:
f(t) = tanh(t)
df(t) = sech(t)^2
ddfoverf(t) = -2 * sech(t)^2
ddfdfoverf2(t) = -2 * csch(t) * sech(t)^3
dddfoverf(t) = 4 * tanh(t) * sech(t)^2 - 2 * csch(t) * sech(t)^3

dddfoverf (generic function with 1 method)

In [4]:
# mode numbers of interest
m = 2
n = 1

# device parameters
R0 = 1

# poloidal wave numbers 
ky = 0.25
km = (m - 1) * ky
k = (m) * ky
kp = (m + 1) * ky

# toroidal wave number
kz = 1 / R0
kn = n * kz

# combined wave numbers
Km2 = km^2 + kn^2
K2 = k^2 + kn^2
Kp2 = kp^2 + kn^2

# Zeta-components
ζz = 1
ζy = 1

# Lundquist number
S = 100

# Alfvénic Mach number
M = S^(-1 / 2)

0.1

In [5]:
function tftearing!(du, u, p, t)
    ψp, φp, ψ,
    ψp1, ψm, ψ1,
    ψm1, φm, φ,
    φ1, φ2, φp1,
    φm1, φ3, φm3 = u
    γ = p[1]
    du[1] = ψp1
    du[2] = φp1
    du[3] = ψ1
    du[4] = Kp2 * ψp + 0.5 * M * S * ψ1 + S * γ * ψp - S * ky * (1 + (1 + m) * f(t)) * φp
    du[5] = ψm1
    du[6] = K2 * ψ + 0.5 * M * S * (ψp1 + ψm1) + S * γ * ψ - S * k * f(t) * φ
    du[7] = Km2 * ψm + 0.5 * M * S * ψ1 + S * γ * ψm - ky * (-1 + (-1 + m) * f(t)) * φm
    du[8] = φm1
    du[9] = φ1
    du[10] = φ2
    du[11] = φ3
    du[12] = (-1 / γ) * (-Kp2 * γ * φ1 + 0.5 * M * (φ3 + (-Kp2 + ζz^2 * ky^2) * φ1) + (-ddfoverf(t) * ψp - Kp2 * ψp + S * (Kp2 * ψp / S + 0.5 * M * ψ1 + ψp * γ - ky * (1 + (1 + m) * f(t)) * φp)) * ky * (1 + (1 + m) * f(t)))
    du[13] = (-1 / γ) * (-Km2 * γ * φ1 + 0.5 * M * (φ3 + (-Km2 + ζz^2 * ky^2) * φ1) + (-ddfoverf(t) * ψm - Km2 * ψm + S * (Km2 * ψm / S + 0.5 * M * ψ1 + ψm * γ - ky * (-1 + (-1 + m) * f(t)) * φm)) * ky * (-1 + (-1 + m) * f(t)))
    du[14] = (-1 / 0.5 * M) * (-(φm3 + Km2 * φm1) * γ - (dddfoverf(t) * ψm + ddfoverf(t) * ψm1 - ddfdfoverf2(t) * ψm - ψm3 + Km2 * ψm1) * ky * (-1 + (-1 + m) * f(t)) + 0.5 * M * (-Km2 + ζz^2 * ky^2) * φ2 + (ddfoverf(t) * ψm + km2 * ψm + S * (-Km2 * ψm / S - 0.5 * M * ψ1 - γ * ψm + ky * (-1 + (-1 + m) * f(t)) * φm)) * ky * (1 - m) * df(t))
    du[15] = (-φp3 + Kp2 * φp1) * γ + ((-ddfdfoverf2(t) * ψp) - ψp3 + (dddfoverf(t) * ψp(t) + ddfoverf(t) * ψp1) + Kp2 * ψp1) * ky * (1 + (1 + m) * f(t)) - 0.5 * M * (-(-φm3 + Km2 * φm1) * γ - ((dddfoverf(t) * ψm + ddfoverf(t) * ψm1) + (-ψm * ddfdfoverf2 - ψm3 + Km2 * ψm1) * ky * (-1 + (-1 + m) * f(t)) + 0.5 * M * (-Km2 + (ky^2) * (ζz^2)) * φ2 + (ψm * ddfoverf(t) + Km2 * ψm + S * ((-Km2 * ψm) / S - 0.5 * M * ψ1 - ψm * γ + ky * (-1 + (-1 + m) * f(t)) * φm)) * ky * (1 - m) * df(t)) / (0.5 * M) + (-Kp2 + (ky^2) * (ζz^2)) * φ2) + (ddfoverf(t) * ψp + Kp2 * ψp + S * ((-Kp2 * ψp) / S - 0.5 * M * ψ1 - ψp * γ + ky * (1 + (1 + m) * f(t)) * φp)) * ky * (1 + m) * df(t)
end

tftearing! (generic function with 1 method)

In [6]:
# typeof(tfmodel)
# @show observed(tfmodel)
# @show unknowns(tfmodel)

In [18]:
jac = calculate_jacobian(tfmodel)

15×15 Matrix{Num}:
                                                                                                                                                                                                                   0  …                0          0
                                                                                                                                                                                                                   0                   0          0
                                                                                                                                                                                                                   0                   0          0
                                                                                                                                                                                                     S*(Kp2 / S + γ)                   0          0
     

In [8]:
mass_matrix = calculate_massmatrix(tfmodel)

15×15 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅

In [9]:
const L = 15.0
tspan = (0.0, L)

(0.0, 15.0)

In [10]:
function bca!(res, u, p)
    # LEFT BOUNDARY (x=0)
    res[1] = u[10]      # ψm'(0)=0,   Dirichlet on right boundary (even)
    res[2] = u[7]       # ψm-1(0)=0, Dirichlet on right boundary (odd)
    res[3] = u[11]      # ψm+1(0)=0, Dirichlet on right boundary (odd)
    res[4] = u[3]       # ϕm(0)=0,    Dirichlet on left boundary (odd)
    res[5] = u[2]       # ϕm-1'(0)=0,  Dirichlet on left boundary (even)
    res[6] = u[6]       # ϕm+1'(0)=0,  Dirichlet on left boundary (even)
    res[7] = u[9] - 1   # ψm(0) = 1,  extra constraint to fix unknown parameter Q
end

function bcb!(res, u, p)
    # RIGHT BOUNDARY (x=L)
    res[1] = u[9]     # ψm(L)=0,    Dirichlet on right boundary
    res[2] = u[7]     # ψm-1(L)=0,  Dirichlet on right boundary
    res[3] = u[11]    # ψm+1(L)=0,  Dirichlet on right boundary
    res[4] = u[3]    # ϕm(L)=0,    Dirichlet on right boundary
    res[5] = u[1]    # ϕm-1(L)=0,  Dirichlet on right boundary
    res[6] = u[5]    # ϕm+1(L)=0,  Dirichlet on right boundary
    res[7] = u[4]    # ϕm'(L)=0,   Neumann on right boundary
    res[8] = u[2]    # ϕm-1'(L)=0, Neumann on right boundary
    res[9] = u[6]    # ϕm+1'(L)=0, Neumann on right boundary
end

bcb! (generic function with 1 method)

In [11]:
u0 = 1e-5 * ones(15)

15-element Vector{Float64}:
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5

In [ ]:
γ_guess = 0.01
fun = BVPFunction(tftearing!, (bca!, bcb!), mass_matrix=mass_matrix, twopoint=Val(true), bcresid_prototype=(zeros(7), zeros(9)))
prob = TwoPointBVProblem(fun, u0, tspan, [γ_guess], fit_parameters=true, )

BVProblem with uType Vector{Float64} and tType Float64. In-place: true
Non-trivial mass matrix: true
timespan: (0.0, 15.0)
u0: 15-element Vector{Float64}:
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5
 1.0e-5

In [13]:
sol = solve(prob, MIRK6(), dt=0.01,
    adaptive=true,
    progress=true,
    verbose=true,
)

LoadError: UndefVarError: `ψm3` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
# print the estimated value of Q which satisfies the BCs
println("γ fitted: ", sol.prob.p[1])

plot(sol, idxs=(0, 9), label=L"ψ(x)")
plot!(sol, idxs=(0, 3), label=L"φ(x)", xlabel="x", legend=:topright)

# plot(sol)